# Classification Notebook - Char 1 Only

This notebook contains machine learning classification algorithms for single character (char 1) epitope prediction.

## Features:
- GPU/CPU training option for all ML algorithms
- Support for multiple propensity scales
- Model saving and evaluation metrics

In [ ]:
# Configuration
import os

# Set device preference: 'gpu' or 'cpu'
DEVICE = 'cpu'  # Change to 'gpu' if GPU is available

# Model directory
model_dir = 'model/'
os.makedirs(model_dir, exist_ok=True)

# Check GPU availability
import tensorflow as tf

if DEVICE == 'gpu':
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        print(f"GPU available: {len(gpus)} GPU(s)")
        for gpu in gpus:
            print(f"  - {gpu}")
    else:
        print("GPU requested but not available. Falling back to CPU.")
        DEVICE = 'cpu'
else:
    print("Using CPU for training")

# Machine Learning Classification Algorithms - Char 1

In [ ]:
import pickle

# Save model to file
def save_pkl(model, name):
    with open(f'{model_dir}{name}.pkl', 'wb') as f:
        pickle.dump(model, f)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import roc_auc_score

def dl(df, X, y, data_test):
    """
    Deep Learning model with GPU/CPU support
    """
    # Configure device
    if DEVICE == 'gpu':
        with tf.device('/GPU:0'):
            return _train_dl(df, X, y, data_test)
    else:
        with tf.device('/CPU:0'):
            return _train_dl(df, X, y, data_test)

def _train_dl(df, X, y, data_test):
    # Drop missing values in the "Position" column
    df = df.dropna(subset=["Position"])

    # Preprocessing data
    X_data = df[X]
    y_data = df[y].values

    # Encode labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y_data)

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_encoded, test_size=data_test, random_state=42)

    # Initialize model
    model = Sequential()
    model.add(Dense(16, activation='relu', input_dim=X_train.shape[1]))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train model
    model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)

    # Predict
    y_pred_proba = model.predict(X_test, verbose=0)

    # Calculate AUC
    auc = roc_auc_score(y_test, y_pred_proba)

    # Evaluate
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)

    # Save model
    model.save(model_dir+'deep_learning_model.h5')

    return accuracy, auc

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score

def nn(df, X, y, data_test):
    """
    Neural Network (MLP) model
    Note: sklearn MLPClassifier uses CPU by default
    """
    # Drop missing values
    df = df.dropna(subset=["Position"])

    # Preprocessing
    X_data = df[X]
    y_data = df[y].values

    # Encode labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y_data)

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_encoded, test_size=data_test, random_state=42)

    # Initialize model
    model = MLPClassifier()

    # Train model
    model.fit(X_train, y_train)

    # Predict
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]

    # Calculate AUC
    auc = roc_auc_score(y_test, y_pred_proba)

    # Save model
    save_pkl(model, 'nnMLP')

    # Calculate accuracy
    accuracy = (y_pred == y_test).mean()

    return accuracy, auc


2023-05-28 08:56:35.389200: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-28 08:56:35.744510: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-28 08:56:35.746364: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-28 08:56:37.885348: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

def rf(df, X, y, data_test):
    """
    Random Forest Classifier
    Note: sklearn RandomForest uses CPU by default
    """
    # Drop missing values
    df = df.dropna(subset=["Position"])

    # Preprocessing
    X_data = df[X]
    y_data = df[y].values

    # Encode labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y_data)

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_encoded, test_size=data_test, random_state=42)

    # Initialize model
    model = RandomForestClassifier()

    # Train model
    model.fit(X_train, y_train)

    # Predict
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]

    # Calculate AUC
    auc = roc_auc_score(y_test, y_pred_proba)

    # Save model
    save_pkl(model, 'RandomForest')

    # Calculate accuracy
    accuracy = (y_pred == y_test).mean()

    return accuracy, auc


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

def dt(df, X, y, data_test):
    """
    Decision Tree Classifier
    """
    # Drop missing values
    df = df.dropna(subset=["Position"])

    # Preprocessing
    X_data = df[X]
    y_data = df[y].values

    # Encode labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y_data)

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_encoded, test_size=data_test, random_state=42)

    # Initialize model
    model = DecisionTreeClassifier()

    # Train model
    model.fit(X_train, y_train)

    # Predict
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]

    # Calculate AUC
    auc = roc_auc_score(y_test, y_pred_proba)

    # Save model
    save_pkl(model, 'DecisionTree')

    # Calculate accuracy
    accuracy = (y_pred == y_test).mean()

    return accuracy, auc


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score

def svm(df, X, y, data_test):
    """
    Support Vector Machine Classifier
    """
    # Drop missing values
    df = df.dropna(subset=["Position"])

    # Preprocessing
    X_data = df[X]
    y_data = df[y].values

    # Encode labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y_data)

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_encoded, test_size=data_test, random_state=42)

    # Initialize model
    model = SVC(probability=True)

    # Train model
    model.fit(X_train, y_train)

    # Predict
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]

    # Calculate AUC
    auc = roc_auc_score(y_test, y_pred_proba)

    # Save model
    save_pkl(model, 'SVM')

    # Calculate accuracy
    accuracy = (y_pred == y_test).mean()

    return accuracy, auc


In [ ]:
from hmmlearn import hmm
from sklearn.metrics import roc_auc_score

def HMM(df, X, y, data_test):
    """
    Hidden Markov Model Classifier
    """
    # Drop missing values
    df = df.dropna(subset=["Position"])

    # Preprocessing
    X_data = df[X]
    y_data = df[y].values

    # Encode labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y_data)

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_encoded, test_size=data_test, random_state=42)

    # Initialize model
    model = hmm.GaussianHMM(n_components=2)

    # Train model
    model.fit(X_train)

    # Predict
    y_pred = model.predict(X_test)
    
    # Get probabilities
    try:
        y_pred_proba = model.predict_proba(X_test)[:, 1]
    except:
        # Fallback if predict_proba not available
        y_pred_proba = y_pred

    # Calculate AUC
    auc = roc_auc_score(y_test, y_pred_proba)

    # Save model
    save_pkl(model, 'hmmmodel')

    # Calculate accuracy
    accuracy = (y_pred == y_test).mean()

    return accuracy, auc


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from hmmlearn import hmm
from sklearn.metrics import roc_auc_score

def HMM(df, X, y,data_test):
    # Drop missing values in the "Position" column
    df = df.dropna(subset=["Position"])

    # Preprocessing data
    X = df[X]
    y = df[y].values

    # Mengubah label menjadi bilangan bulat
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    # Membagi data menjadi data latih dan data uji
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=data_test, random_state=42)

    # Menginisialisasi model HMM dengan Gaussian emissions
    model = hmm.GaussianHMM(n_components=2)

    # Melatih model HMM dengan data latih
    model.fit(X_train)

    # Memprediksi label pada data uji
    y_pred = model.predict(X_test)

    # Mengubah label hasil prediksi menjadi label awal
    y_pred_label = label_encoder.inverse_transform(y_pred)

    # Menghitung probabilitas prediksi pada data uji
    y_pred_proba = model.predict_proba(X_test)[:, 1]

    # Menghitung nilai AUC (Area Under Curve)
    auc = roc_auc_score(y_test, y_pred_proba)

    save_pkl(model,'hmmmodel')

    # Menghitung akurasi
    accuracy = (y_pred_label == label_encoder.inverse_transform(y_test)).mean()

    # Mengembalikan akurasi dan nilai ROC
    return accuracy, auc


# Penerapan

In [9]:
import pandas as pd

In [10]:
df1 = pd.read_csv('dataset/dataset_type_2_vers2_hidropobicity.csv')
df2 = pd.read_csv('dataset/dataset_type_2_char2_vers2.csv')

In [16]:
42/3

14.0

# Results Summary


In [ ]:
# Display summary statistics
print("Summary Statistics:")
print(dtest.groupby(['algoritm', 'test_size'])[['accuracy', 'auc']].mean())
print("\nBest Results:")
print(dtest.loc[dtest['auc'].idxmax()])


In [11]:
dtest = pd.DataFrame()
mod = ['hmm','nn','dl','dt','rf','svm']
#prop_scale = ['hoop_woods','emini','parker','levitt']
prop_scale = ['Kyte-Doolittle', 'Hopp-Woods', 'Cornette', 'Eisenberg', 'Rose', 'Janin', 'Engelman GES']
dt_test = [0.1,0.2,0.3]
mod4x = mod*len(prop_scale)
prop_scale6x = prop_scale*len(mod)
dtest['algoritm'] = mod4x
dtest['prop_scale'] = prop_scale6x
dtest['accuracy'] = 0.0
dtest['auc'] = 0.0
dtest['n_amino_acids'] = 1
dtest['test_size'] = 0.0
dtest = pd.concat([dtest,dtest,dtest])
dtest = dtest.reset_index(drop=True)
#dtest = dtest.sample(frac=1).reset_index(drop=True)
for i in mod:
    dtest.loc[dtest['algoritm'] == i, 'test_size'] = int(len(dtest[dtest['algoritm'] == i])/len(dt_test))*dt_test

In [12]:
len(dtest)

126

# Penerapan Char 1

In [13]:
dtest = pd.read_csv('result_of_classification.csv')

In [14]:
begin = dtest[dtest['accuracy'] == 0.00]

In [15]:
begin.index[0]

101

In [16]:
y = 'label'
for i in range(begin.index[0],len(dtest)):
    X = ['Position z-score']

    if(dtest['algoritm'][i] == 'hmm'):
        X.append(dtest['prop_scale'][i])
        acc, auc = HMM(df1,X, y,dtest['test_size'][i])
    
    if(dtest['algoritm'][i] == 'nn'):
        X.append(dtest['prop_scale'][i])
        acc, auc = nn(df1,X, y,dtest['test_size'][i])
    
    if(dtest['algoritm'][i] == 'dl'):
        X.append(dtest['prop_scale'][i])
        acc, auc = dl(df1,X, y,dtest['test_size'][i])
    
    if(dtest['algoritm'][i] == 'dt'):
        X.append(dtest['prop_scale'][i])
        acc, auc = dt(df1,X, y,dtest['test_size'][i])
    
    if(dtest['algoritm'][i] == 'rf'):
        X.append(dtest['prop_scale'][i])
        acc, auc = rf(df1,X, y,dtest['test_size'][i])

    if(dtest['algoritm'][i] == 'svm'):
        X.append(dtest['prop_scale'][i])
        acc, auc = svm(df1,X, y,dtest['test_size'][i])

    dtest['accuracy'][i] = acc
    dtest['auc'][i] = auc

    dtest.to_csv('result_of_classification.csv',index=False)

/tmp/ipykernel_25624/460075434.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['accuracy'][i] = acc
/tmp/ipykernel_25624/460075434.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['auc'][i] = auc
/tmp/ipykernel_25624/460075434.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['accuracy'][i] = acc
/tmp/ipykernel_25624/460075434.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

3238/3238 [==============================] - 4s 1ms/step


/tmp/ipykernel_25624/460075434.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['accuracy'][i] = acc
/tmp/ipykernel_25624/460075434.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['auc'][i] = auc
/tmp/ipykernel_25624/460075434.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['accuracy'][i] = acc
/tmp/ipykernel_25624/460075434.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

1080/1080 [==============================] - 1s 1ms/step


/tmp/ipykernel_25624/460075434.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['accuracy'][i] = acc
/tmp/ipykernel_25624/460075434.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['auc'][i] = auc
/tmp/ipykernel_25624/460075434.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['accuracy'][i] = acc
/tmp/ipykernel_25624/460075434.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

2159/2159 [==============================] - 2s 1ms/step


/tmp/ipykernel_25624/460075434.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['accuracy'][i] = acc
/tmp/ipykernel_25624/460075434.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['auc'][i] = auc
/tmp/ipykernel_25624/460075434.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['accuracy'][i] = acc
/tmp/ipykernel_25624/460075434.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

3238/3238 [==============================] - 6s 2ms/step


/tmp/ipykernel_25624/460075434.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['accuracy'][i] = acc
/tmp/ipykernel_25624/460075434.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['auc'][i] = auc
/tmp/ipykernel_25624/460075434.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['accuracy'][i] = acc
/tmp/ipykernel_25624/460075434.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

# Penerapan Char 2

In [15]:
df2

,amino 1,amino 2,Position,label,k,hoop_woods 1,emini 1,parker 1,levitt 1,hoop_woods 2,emini 2,parker 2,levitt 2
0,S,V,1175,EE,2,-2.0,0.92,-0.25,0.79,1.0,1.02,1.24,1.30
1,V,V,1177,EE,2,1.0,1.02,1.24,1.30,1.0,1.02,1.24,1.30
2,V,N,1179,EE,2,1.0,1.02,1.24,1.30,-3.0,0.92,-0.12,0.68
3,N,I,1181,EE,2,-3.0,0.92,-0.12,0.68,1.0,1.02,1.61,1.18
4,I,Q,1183,EE,2,1.0,1.02,1.61,1.18,-3.0,0.92,-0.71,0.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...
323081,L,L,887,..,2,1.0,1.02,1.30,1.21,1.0,1.02,1.30,1.21
323082,L,A,889,..,2,1.0,1.02,1.30,1.21,1.0,0.92,1.53,1.23
323083,A,G,891,..,2,1.0,0.92,1.53,1.23,0.0,0.74,0.74,1.15
323084,G,T,893,..,2,0.0,0.74,0.74,1.15,-2.0,0.92,-0.18,0.82


In [18]:
X = ['Position', 'hoop_woods 1', 'hoop_woods 2']
y = 'label'
acc, auc = nn(df2,X, y)
print(acc)

0.6512272122318858


In [23]:
mod = ['hmm','nn','dl','dt','rf','svm']
prop_scale = ['hoop_woods','emini','parker','levitt']
mod4x = mod*len(prop_scale)
prop_scale6x = prop_scale*len(mod)
dtest['algoritm'] = mod4x
dtest['prop_scale'] = prop_scale6x
dtest['accuracy'] = 0.0
dtest['auc'] = 0.0
dtest['n_amino_acids'] = 2

In [24]:
dtest

,algoritm,prop_scale,accuracy,auc,n_amino_acids
0,hmm,hoop_woods,0.0,0.0,2
1,nn,emini,0.0,0.0,2
2,dl,parker,0.0,0.0,2
3,dt,levitt,0.0,0.0,2
4,rf,hoop_woods,0.0,0.0,2
5,svm,emini,0.0,0.0,2
6,hmm,parker,0.0,0.0,2
7,nn,levitt,0.0,0.0,2
8,dl,hoop_woods,0.0,0.0,2
9,dt,emini,0.0,0.0,2


In [ ]:
y = 'label'
for i in range(len(dtest)):
    X = ['Position']

    if(dtest['algoritm'][i] == 'hmm'):
        X.append(dtest['prop_scale'][i]+" 1")
        X.append(dtest['prop_scale'][i]+" 2")
        acc, auc = HMM(df2,X, y)
    
    if(dtest['algoritm'][i] == 'nn'):
        X.append(dtest['prop_scale'][i]+" 1")
        X.append(dtest['prop_scale'][i]+" 2")
        acc, auc = nn(df2,X, y)
    
    if(dtest['algoritm'][i] == 'dl'):
        X.append(dtest['prop_scale'][i]+" 1")
        X.append(dtest['prop_scale'][i]+" 2")
        acc, auc = dl(df2,X, y)
    
    if(dtest['algoritm'][i] == 'dt'):
        X.append(dtest['prop_scale'][i]+" 1")
        X.append(dtest['prop_scale'][i]+" 2")
        acc, auc = dt(df2,X, y)
    
    if(dtest['algoritm'][i] == 'rf'):
        X.append(dtest['prop_scale'][i]+" 1")
        X.append(dtest['prop_scale'][i]+" 2")
        acc, auc = rf(df2,X, y)

    if(dtest['algoritm'][i] == 'svm'):
        X.append(dtest['prop_scale'][i]+" 1")
        X.append(dtest['prop_scale'][i]+" 2")
        acc, auc = svm(df2,X, y)

    dtest['accuracy'][i] = acc
    dtest['auc'][i] = auc

    dtest.to_csv('result_of_classification_char2.csv',index=False)

In [16]:
# dijumlahkan propensity scale nya
y = 'label'
for i in range(len(dtest)):
    X = ['Position']

    if(dtest['algoritm'][i] == 'hmm'):
        X.append(dtest['prop_scale'][i])
        acc, auc = HMM(df2,X, y)
    
    if(dtest['algoritm'][i] == 'nn'):
        X.append(dtest['prop_scale'][i])
        acc, auc = nn(df2,X, y)
    
    if(dtest['algoritm'][i] == 'dl'):
        X.append(dtest['prop_scale'][i])
        acc, auc = dl(df2,X, y)
    
    if(dtest['algoritm'][i] == 'dt'):
        X.append(dtest['prop_scale'][i])
        acc, auc = dt(df2,X, y)
    
    if(dtest['algoritm'][i] == 'rf'):
        X.append(dtest['prop_scale'][i])
        acc, auc = rf(df2,X, y)

    if(dtest['algoritm'][i] == 'svm'):
        X.append(dtest['prop_scale'][i])
        acc, auc = svm(df2,X, y)

    dtest['accuracy'][i] = acc
    dtest['auc'][i] = auc

    dtest.to_csv('result_of_classification_char2_vers2.csv',index=False)

/tmp/ipykernel_6605/1599508817.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['accuracy'][i] = acc
/tmp/ipykernel_6605/1599508817.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['auc'][i] = auc
/tmp/ipykernel_6605/1599508817.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['accuracy'][i] = acc
/tmp/ipykernel_6605/1599508817.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

2020/2020 [==============================] - 5s 3ms/step


/tmp/ipykernel_6605/1599508817.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['accuracy'][i] = acc
/tmp/ipykernel_6605/1599508817.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['auc'][i] = auc
/tmp/ipykernel_6605/1599508817.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['accuracy'][i] = acc
/tmp/ipykernel_6605/1599508817.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

2020/2020 [==============================] - 2s 1ms/step


/tmp/ipykernel_6605/1599508817.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['accuracy'][i] = acc
/tmp/ipykernel_6605/1599508817.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['auc'][i] = auc
/tmp/ipykernel_6605/1599508817.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['accuracy'][i] = acc
/tmp/ipykernel_6605/1599508817.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

2020/2020 [==============================] - 4s 2ms/step


/tmp/ipykernel_6605/1599508817.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['accuracy'][i] = acc
/tmp/ipykernel_6605/1599508817.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['auc'][i] = auc
/tmp/ipykernel_6605/1599508817.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['accuracy'][i] = acc
/tmp/ipykernel_6605/1599508817.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

2020/2020 [==============================] - 4s 2ms/step


/tmp/ipykernel_6605/1599508817.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['accuracy'][i] = acc
/tmp/ipykernel_6605/1599508817.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['auc'][i] = auc
/tmp/ipykernel_6605/1599508817.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtest['accuracy'][i] = acc
/tmp/ipykernel_6605/1599508817.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice